### Initial BeautifulSoup4 Demo:
- [Site being scraped](https://realpython.github.io/fake-jobs/)
- Want to find better site, learning the library as of right now.

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [31]:
def receive_structure():
    url = "https://realpython.github.io/fake-jobs/"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    with open("home-structure.html", "w") as file:
        file.write(soup.prettify())

In [ ]:
def extract_structure_of_application_page(data):
    link = data["link"][1] # only grab the first link because they are all the same
    response = requests.get(link)
    soup = BeautifulSoup(response.text, "html.parser")

    with open("application-structure.html", "w") as file:
        file.write(soup.prettify())

In [33]:
def load_data():
    url = "https://realpython.github.io/fake-jobs/"

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    collective_job_data = []

    job_cards = soup.find_all("div", class_="card-content")

    for card in job_cards:
        job_data = {}
        if card.find("time"):
            job_data["date"] = card.find("time").text.strip()
        if card.find("h2", class_="title"):
            job_data["title"] = card.find("h2", class_="title").text.strip()
        if card.find("p", class_="location"):
            job_data["location"] = card.find("p", class_="location").text.strip()
        if card.find("h3", class_="company"):
            job_data["company"] = card.find("h3", class_="subtitle is-6 company").text.strip()
        if card.find_all("a")[1]["href"]:
            job_data["link"] = card.find_all("a")[1]["href"]
        collective_job_data.append(job_data)

    return pd.DataFrame(collective_job_data)

In [34]:
def extract_description_from_application_link(data):
    for index, row in data.iterrows():
        response = requests.get(row["link"])
        soup = BeautifulSoup(response.text, "html.parser")
        if soup.find("p", class_="content"):
            data.loc[index, "description"] = soup.find("div", class_="content").text.strip()
    return data

In [35]:
def write_to_csv(data):
    data.to_csv("jobs.csv", index=False)

In [ ]:
data = load_data()

receive_structure()
extract_structure_of_application_page(data)

write_to_csv(data)

In [38]:
data.head()

,date,title,location,company,link
0,2021-04-08,Senior Python Developer,"Stewartbury, AA","Payne, Roberts and Davis",https://realpython.github.io/fake-jobs/jobs/se...
1,2021-04-08,Energy engineer,"Christopherville, AA",Vasquez-Davidson,https://realpython.github.io/fake-jobs/jobs/en...
2,2021-04-08,Legal executive,"Port Ericaburgh, AA","Jackson, Chambers and Levy",https://realpython.github.io/fake-jobs/jobs/le...
3,2021-04-08,Fitness centre manager,"East Seanview, AP",Savage-Bradley,https://realpython.github.io/fake-jobs/jobs/fi...
4,2021-04-08,Product manager,"North Jamieview, AP",Ramirez Inc,https://realpython.github.io/fake-jobs/jobs/pr...
